# Fintel

### Generating GPU-Accelerated Quantitative Finance Signals

In [74]:
# Install some magic to make c++ programs look nice!
!wget -O cpp_plugin.py https://gist.github.com/akshaykhadse/7acc91dd41f52944c6150754e5530c4b/raw/cpp_plugin.py
%load_ext cpp_plugin

--2023-04-30 22:37:38--  https://gist.github.com/akshaykhadse/7acc91dd41f52944c6150754e5530c4b/raw/cpp_plugin.py
Resolving gist.github.com (gist.github.com)... 20.205.243.166
Connecting to gist.github.com (gist.github.com)|20.205.243.166|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://gist.githubusercontent.com/akshaykhadse/7acc91dd41f52944c6150754e5530c4b/raw/cpp_plugin.py [following]
--2023-04-30 22:37:38--  https://gist.githubusercontent.com/akshaykhadse/7acc91dd41f52944c6150754e5530c4b/raw/cpp_plugin.py
Resolving gist.githubusercontent.com (gist.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to gist.githubusercontent.com (gist.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2730 (2.7K) [text/plain]
Saving to: ‘cpp_plugin.py’

cpp_plugin.py       100%[===================>]   2.67K  --.-KB/s    in 0s      

2023-04-30 22:37:3

In [75]:
# make sure CUDA is installed
!nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2022 NVIDIA Corporation
Built on Wed_Sep_21_10:33:58_PDT_2022
Cuda compilation tools, release 11.8, V11.8.89
Build cuda_11.8.r11.8/compiler.31833905_0


In [76]:
# make sure you have a GPU runtime (if this fails go to runtime -> change runtime type)
!nvidia-smi

Sun Apr 30 22:37:39 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.85.12    Driver Version: 525.85.12    CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   43C    P8     9W /  70W |      0MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

#### Download stock price data

In [77]:
!pip install yfinance > /dev/null 2>&1

In [78]:
import yfinance as yf

# Define the stock symbol and the date range for which you want to download the data
symbol = "MSFT"
start_date = "1986-03-12"
end_date = "2023-01-01"

# Download historical stock data using yfinance
stock_data = yf.download(symbol, start=start_date, end=end_date)

# Save the downloaded data as a CSV file
csv_filename = "stock_prices.csv"
stock_data.to_csv(csv_filename)

print(f"Historical stock data for {symbol} saved to {csv_filename}")

[*********************100%***********************]  1 of 1 completed
Historical stock data for MSFT saved to stock_prices.csv




## The Interface

In [79]:
#@title The output of the cell is formatted text double click the title to show or hide the raw code you can edit!
%%cpp -n IFintel.h -s xcode

#ifndef IFintel_H
#define IFintel_H

#include <vector>
#include <string>

class IFintel
{
public:
    struct DataPoint
    {
        std::string date;
        float open;
        float high;
        float low;
        float close;
    };
    virtual ~IFintel(){};
    virtual std::vector<float> calculate_sma(const std::vector<DataPoint>& data, int window_size) = 0;
    virtual std::vector<float> calculate_ema(const std::vector<DataPoint>& data, int window_size) = 0;
    virtual std::vector<float> calculate_rsi(const std::vector<DataPoint>& data, int window_size) = 0;
};

#endif

## The CPU Implementation

In [80]:
#@title Header File
%%cpp -n CPUFintel.h -s xcode

#ifndef CPUFINTEL_H
#define CPUFINTEL_H

#include "IFintel.h"

class CPUFintel : public IFintel
{
public:
    CPUFintel(){};
    std::vector<float> calculate_sma(const std::vector<DataPoint>& data, int window_size);
    std::vector<float> calculate_ema(const std::vector<DataPoint>& data, int window_size);
    std::vector<float> calculate_rsi(const std::vector<DataPoint>& data, int window_size);
    ~CPUFintel(){};
};

#endif

In [81]:
#@title Implementation
%%cpp -n CPUFintel.cpp -s xcode

#include "CPUFintel.h"

std::vector<float> CPUFintel::calculate_rsi(const std::vector<DataPoint>& data, int window_size) {
    std::vector<float> rsi(data.size());

    float up_sum = 0;
    float down_sum = 0;

    for (int i = 1; i < window_size; i++) {
        float delta = data[i].close - data[i-1].close;
        if (delta > 0) {
            up_sum += delta;
        } else {
            down_sum -= delta;
        }
    }

    float avg_up = up_sum / window_size;
    float avg_down = down_sum / window_size;
    float rs = avg_up / avg_down;
    rsi[window_size] = 100 - (100 / (1 + rs));

    for (unsigned int i = window_size + 1; i < data.size(); i++) {
        float delta = data[i].close - data[i-1].close;
        if (delta > 0) {
            up_sum += delta;
            down_sum -= 0;
        } else {
            down_sum -= delta;
            up_sum -= 0;
        }

        avg_up = up_sum / window_size;
        avg_down = down_sum / window_size;
        rs = avg_up / avg_down;
        rsi[i] = 100 - (100 / (1 + rs));
    }

    return rsi;
}

std::vector<float> CPUFintel::calculate_ema(const std::vector<DataPoint>& data, int window_size)
{
    std::vector<float> ema(data.size());

    float multiplier = 2.0f / (window_size + 1);
    ema[0] = data[0].close;

    for (unsigned int i = 1; i < data.size(); i++) {
        ema[i] = (data[i].close - ema[i - 1]) * multiplier + ema[i - 1];
    }

    return ema;
}

std::vector<float> CPUFintel::calculate_sma(const std::vector<DataPoint>& data, int window_size)
{
    std::vector<float> sma(data.size());

    float sum = 0;
    for (int i = 0; i < window_size; i++) {
        sum += data[i].close;
        sma[i] = sum / (i + 1);
    }

    for (unsigned int i = window_size; i < data.size(); i++) {
        sum += data[i].close - data[i - window_size].close;
        sma[i] = sum / window_size;
    }
    
    return sma;
}

## The GPU Implementation

In [82]:
#@title Header File
%%cpp -n GPUFintel.h -s xcode

#ifndef GPUFINTEL_H
#define GPUFINTEL_H

#include "IFintel.h"

class GPUFintel : public IFintel
{
public:
    GPUFintel(){};
    std::vector<float> calculate_sma(const std::vector<DataPoint>& data, int window_size);
    std::vector<float> calculate_ema(const std::vector<DataPoint>& data, int window_size);
    std::vector<float> calculate_rsi(const std::vector<DataPoint>& data, int window_size);
    ~GPUFintel(){};
 
private:
    template <typename KernelFunc> std::vector<float> calculate_indicator(const std::vector<DataPoint>& data, int window_size, KernelFunc kernel);

};

#endif

In [83]:
#@title Header File
%%cpp -n gpu_helpers.h -s xcode

#ifndef GPU_HELPERS_H
#define GPU_HELPERS_H

#include <cuda_runtime.h>

__device__ void sma_function(float *input, float *output, int window_size, int data_size, int idx);
__device__ void ema_function(float *input, float *output, int window_size, int data_size, int idx);
__device__ void rsi_function(float *input, float *output, int window_size, int data_size, int idx);

#endif

In [84]:
#@title Implementation
%%writefile GPUFintel.cu

#include "GPUFintel.h"
#include "gpu_helpers.h"

#include <cuda_runtime.h>
#include <device_launch_parameters.h>

__global__ void sma_kernel(float *input, float *output, int window_size, int data_size) {
    sma_function(input, output, window_size, data_size, threadIdx.x + blockIdx.x * blockDim.x);
}

__global__ void ema_kernel(float *input, float *output, int window_size, int data_size) {
    ema_function(input, output, window_size, data_size, threadIdx.x + blockIdx.x * blockDim.x);
}

__global__ void rsi_kernel(float *input, float *output, int window_size, int data_size) {
    rsi_function(input, output, window_size, data_size, threadIdx.x + blockIdx.x * blockDim.x);
}

template <typename KernelFunc>
std::vector<float> GPUFintel::calculate_indicator(const std::vector<DataPoint>& data, int window_size, KernelFunc kernel)
{
    int data_size = data.size();
    float *h_input = new float[data_size];
    float *h_output = new float[data_size];

    for (int i = 0; i < data_size; i++) {
        h_input[i] = data[i].close;
    }

    float *d_input, *d_output;
    cudaMalloc((void **)&d_input, data_size * sizeof(float));
    cudaMalloc((void **)&d_output, data_size * sizeof(float));

    cudaMemcpy(d_input, h_input, data_size * sizeof(float), cudaMemcpyHostToDevice);

    int blockSize = 256;
    int gridSize = (data_size + blockSize - 1) / blockSize;

    kernel<<<gridSize, blockSize>>>(d_input, d_output, window_size, data_size);

    cudaMemcpy(h_output, d_output, data_size * sizeof(float), cudaMemcpyDeviceToHost);

    cudaFree(d_input);
    cudaFree(d_output);

    std::vector<float> result(h_output, h_output + data_size);

    delete[] h_input;
    delete[] h_output;

    return result;
}

std::vector<float> GPUFintel::calculate_sma(const std::vector<DataPoint>& data, int window_size)
{
    return calculate_indicator(data, window_size, sma_kernel);
}

std::vector<float> GPUFintel::calculate_ema(const std::vector<DataPoint>& data, int window_size)
{
    return calculate_indicator(data, window_size, ema_kernel);
}

std::vector<float> GPUFintel::calculate_rsi(const std::vector<DataPoint>& data, int window_size)
{
    return calculate_indicator(data, window_size, rsi_kernel);
}


Overwriting GPUFintel.cu


In [85]:
#@title Implementation
%%writefile gpu_helpers.cu

#include "gpu_helpers.h"


__device__ void sma_function(float *input, float *output, int window_size, int data_size, int idx) {
    if (idx < data_size) {
        float sum = 0.0f;
        int count = 0;
        for (int i = idx - window_size + 1; i <= idx; i++) {
            if (i >= 0) {
                sum += input[i];
                count++;
            }
        }
        output[idx] = sum / count;
    }
}

__device__ void ema_function(float *input, float *output, int window_size, int data_size, int idx) {
    if (idx < data_size) {
        float alpha = 2.0f / (window_size + 1.0f);
        if (idx == 0) {
            output[idx] = input[idx];
        } else {
            output[idx] = alpha * input[idx] + (1.0f - alpha) * output[idx - 1];
        }
    }
}

__device__ void rsi_function(float *input, float *output, int window_size, int data_size, int idx) {
    if (idx >= window_size && idx < data_size) {
        float gain_sum = 0.0f;
        float loss_sum = 0.0f;
        for (int i = idx - window_size + 1; i <= idx; i++) {
            float diff = input[i] - input[i-1];
            if (diff > 0) {
                gain_sum += diff;
            } else if (diff < 0) {
                loss_sum += abs(diff);
            }
        }
        float rs = gain_sum / window_size / (loss_sum / window_size);
        output[idx] = 100.0f - 100.0f / (1.0f + rs);
    }
}

Overwriting gpu_helpers.cu


In [86]:
#@title Header File
%%cpp -n TimedGPUFintel.h -s xcode

#ifndef TIMEDGPUFINTEL_H
#define TIMEDGPUFINTEL_H

#include "IFintel.h"

class TimedGPUFintel : public IFintel {
public:
    TimedGPUFintel(){};
    std::vector<float> calculate_sma(const std::vector<DataPoint>& data, int window_size);
    std::vector<float> calculate_ema(const std::vector<DataPoint>& data, int window_size);
    std::vector<float> calculate_rsi(const std::vector<DataPoint>& data, int window_size);
    ~TimedGPUFintel(){};

private:
  template <typename KernelFunc, typename KernelFunc2> std::vector<float> calculate_indicator(const std::vector<DataPoint>& data, int window_size, KernelFunc kernel, KernelFunc2 kernel_timing);
};

#endif


In [87]:
#@title Implementation
%%writefile TimedGPUFintel.cu

#include "gpu_helpers.h"
#include "TimedGPUFintel.h" 

#include <iostream>
#include <numeric>
#include <cuda_runtime.h>
#include <device_launch_parameters.h>

#define time_delta_us_timespec(start,end) (1e6*static_cast<double>(end.tv_sec - start.tv_sec)+1e-3*static_cast<double>(end.tv_nsec - start.tv_nsec))
#define TEST_ITERS_GLOBAL 100

void printStats(std::vector<double> *times){
   double sum = std::accumulate(times->begin(), times->end(), 0.0);
   double mean = sum/static_cast<double>(times->size());
   std::cout << "Mean: " << mean << "\n";
}

__global__ void sma_kernel_notime(float *input, float *output, int window_size, int data_size) {
    sma_function(input, output, window_size, data_size, threadIdx.x + blockIdx.x * blockDim.x);
}

__global__ void ema_kernel_notime(float *input, float *output, int window_size, int data_size) {
    ema_function(input, output, window_size, data_size, threadIdx.x + blockIdx.x * blockDim.x);
}

__global__ void rsi_kernel_notime(float *input, float *output, int window_size, int data_size) {
    rsi_function(input, output, window_size, data_size, threadIdx.x + blockIdx.x * blockDim.x);
}


__global__
void sma_kernel_timing(float *input, float *output, int window_size, int data_size, int TEST_ITERS) {
  	for(int iter = 0; iter < TEST_ITERS; iter++){
      sma_function(input, output, window_size, data_size, threadIdx.x + blockIdx.x * blockDim.x);
    }
}

__global__
void ema_kernel_timing(float *input, float *output, int window_size, int data_size, int TEST_ITERS) {
  	for(int iter = 0; iter < TEST_ITERS; iter++){
      ema_function(input, output, window_size, data_size, threadIdx.x + blockIdx.x * blockDim.x);
    }
}

__global__
void rsi_kernel_timing(float *input, float *output, int window_size, int data_size, int TEST_ITERS) {
  	for(int iter = 0; iter < TEST_ITERS; iter++){
      rsi_function(input, output, window_size, data_size, threadIdx.x + blockIdx.x * blockDim.x);
    }
}

template <typename KernelFunc>
__host__ void end_to_end_kernel_with_memory_test(float *h_input, float *h_output, float *d_input, float *d_output, int window_size, int data_size, KernelFunc kernel){
    cudaMemcpy(d_input, h_input, data_size * sizeof(float), cudaMemcpyHostToDevice);

    int blockSize = 256;
    int gridSize = (data_size + blockSize - 1) / blockSize;

    kernel<<<gridSize, blockSize>>>(d_input, d_output, window_size, data_size);

    cudaMemcpy(h_output, d_output, data_size * sizeof(float), cudaMemcpyDeviceToHost);
}

template <typename KernelFunc>
__host__ void end_to_end_kernel_no_memory_test(float *d_input, float *d_output, int window_size, int data_size, KernelFunc kernel){
  int blockSize = 256;
  int gridSize = (data_size + blockSize - 1) / blockSize;
	kernel<<<gridSize, blockSize>>>(d_input, d_output, window_size, data_size);
}

template <int TEST_ITERS, typename KernelFunc, typename KernelFunc2>
__host__
void test(int SWEEP_PARAMETER, float *h_input, float *h_output, float *d_input, float *d_output, int window_size, int data_size, KernelFunc kernel, KernelFunc2 kernel_timing){
      int blockSize = 256;
      int gridSize = (data_size + blockSize - 1) / blockSize;
   	  
      struct timespec start, end;
      std::vector<double> times = {};

      if (SWEEP_PARAMETER == 0) {
        clock_gettime(CLOCK_MONOTONIC,&start);
        kernel_timing<<<gridSize, blockSize>>>(d_input, d_output, window_size, data_size, TEST_ITERS);
        clock_gettime(CLOCK_MONOTONIC,&end);
        printf("[N:1]: Ignore Me -- Initial Slow One: %f\n",time_delta_us_timespec(start,end)/static_cast<double>(TEST_ITERS));

        clock_gettime(CLOCK_MONOTONIC,&start);
        kernel_timing<<<gridSize, blockSize>>>(d_input, d_output, window_size, data_size, TEST_ITERS);
        clock_gettime(CLOCK_MONOTONIC,&end);
        printf("[N:1]: Kernel Under Test: %f\n",time_delta_us_timespec(start,end)/static_cast<double>(TEST_ITERS));
      }
      else {
        for(int iter = 0; iter < TEST_ITERS; iter++){
          clock_gettime(CLOCK_MONOTONIC,&start);
          end_to_end_kernel_with_memory_test(h_input, h_output, d_input, d_output, window_size, data_size, kernel);
          clock_gettime(CLOCK_MONOTONIC,&end);
          times.push_back(time_delta_us_timespec(start,end));
        }
        printf("[N:%d]: Ignore Me -- Initial Slow One: ",SWEEP_PARAMETER); printStats(&times); times.clear();
    
        for(int iter = 0; iter < TEST_ITERS; iter++){
          clock_gettime(CLOCK_MONOTONIC,&start);
          end_to_end_kernel_with_memory_test(h_input, h_output, d_input, d_output, window_size, data_size, kernel);
          clock_gettime(CLOCK_MONOTONIC,&end);
          times.push_back(time_delta_us_timespec(start,end));
        }
        printf("[N:%d]: Kernel Under Test - With Memory: ",SWEEP_PARAMETER); printStats(&times); times.clear();
    
        for(int iter = 0; iter < TEST_ITERS; iter++){
          cudaMemcpy(d_input, h_input, data_size * sizeof(float), cudaMemcpyHostToDevice);
          clock_gettime(CLOCK_MONOTONIC,&start);
          end_to_end_kernel_no_memory_test(d_input, d_output, window_size, data_size, kernel);
          clock_gettime(CLOCK_MONOTONIC,&end);
          cudaMemcpy(h_output, d_output, data_size * sizeof(float), cudaMemcpyDeviceToHost);
          times.push_back(time_delta_us_timespec(start,end));
        }
        printf("[N:%d]: Kernel Under Test - Compute Only: ",SWEEP_PARAMETER); printStats(&times); times.clear();
      }
}

template <typename KernelFunc, typename KernelFunc2>
std::vector<float> TimedGPUFintel::calculate_indicator(const std::vector<DataPoint>& data, int window_size, KernelFunc kernel, KernelFunc2 kernel_timing)
{
    int data_size = data.size();
    float *h_input = new float[data_size];
    float *h_output = new float[data_size];

    for (int i = 0; i < data_size; i++) {
        h_input[i] = data[i].close;
    }

    float *d_input, *d_output;
    cudaMalloc((void **)&d_input, data_size * sizeof(float));
    cudaMalloc((void **)&d_output, data_size * sizeof(float));

    test<TEST_ITERS_GLOBAL>(data_size, h_input, h_output, d_input, d_output, window_size, data_size, kernel, kernel_timing);

    cudaFree(d_input);
    cudaFree(d_output);

    std::vector<float> result(h_output, h_output + data_size);

    delete[] h_input;
    delete[] h_output;

    return result;
}

std::vector<float> TimedGPUFintel::calculate_sma(const std::vector<DataPoint>& data, int window_size)
{
    return calculate_indicator(data, window_size, sma_kernel_notime, sma_kernel_timing);
}

std::vector<float> TimedGPUFintel::calculate_ema(const std::vector<DataPoint>& data, int window_size)
{
    return calculate_indicator(data, window_size, ema_kernel_notime, ema_kernel_timing);
}

std::vector<float> TimedGPUFintel::calculate_rsi(const std::vector<DataPoint>& data, int window_size)
{
    return calculate_indicator(data, window_size, rsi_kernel_notime, rsi_kernel_timing);
}

Overwriting TimedGPUFintel.cu


## Controller



In [88]:
#@title Header
%%cpp -n Controller.h -s xcode

#ifndef CONTROLLER_H
#define CONTROLLER_H

class Controller
{
private:
    std::shared_ptr<IFintel> fintel;

public:
    Controller(std::shared_ptr<IFintel> fintel_) : fintel(fintel_){};
    void benchmark(std::string indicator);
    std::vector<IFintel::DataPoint> getData(const std::string& filename);
    std::vector<IFintel::DataPoint> getFakeData(int data_size);

};

#endif

In [89]:
#@title Implementation
%%cpp -n controller.cpp -s xcode

#include <iostream>
#include <memory>
#include <vector>
#include <fstream>
#include <sstream>
#include <random>

#include "IFintel.h"
#include "CPUFintel.h"
#include "GPUFintel.h"
#include "TimedGPUFintel.h"
#include "Controller.h"

#define time_delta_us_timespec(start,end) (1e6*static_cast<double>(end.tv_sec - start.tv_sec)+1e-3*static_cast<double>(end.tv_nsec - start.tv_nsec))


void Controller::benchmark(std::string indicator) {
    int window_size = 10;
    struct timespec start, end;
    int num = 100;
    {
      std::vector<IFintel::DataPoint> data = getFakeData(num);
      if (indicator == "sma")
        std::vector<float> sma = fintel->calculate_rsi(data, window_size);
      else if (indicator == "ema")
        std::vector<float> ema = fintel->calculate_rsi(data, window_size);
      else if (indicator == "rsi")
        std::vector<float> rsi = fintel->calculate_rsi(data, window_size);
    }
    while (num <= 10000000) {
      std::vector<IFintel::DataPoint> data = getFakeData(num);
      clock_gettime(CLOCK_MONOTONIC,&start);
      if (indicator == "sma")
        std::vector<float> sma = fintel->calculate_rsi(data, window_size);
      else if (indicator == "ema")
        std::vector<float> ema = fintel->calculate_rsi(data, window_size);
      else if (indicator == "rsi")
        std::vector<float> rsi = fintel->calculate_rsi(data, window_size);
      clock_gettime(CLOCK_MONOTONIC,&end);
      printf("[N:%d] E2E Timing: %f\n", num, time_delta_us_timespec(start,end)/static_cast<double>(1));
      num *= 2;
    }
    return;
}

std::vector<IFintel::DataPoint> Controller::getData(const std::string& filename) {
    std::vector<IFintel::DataPoint> data;
    std::ifstream file(filename);
    std::string line;
    
    if (file.is_open()) {
        std::getline(file, line);
        while (getline(file, line)) {
            std::stringstream ss(line);

            std::string date, open, high, low, close, adj_close, volume;
            getline(ss, date, ',');
            getline(ss, open, ',');
            getline(ss, high, ',');
            getline(ss, low, ',');
            getline(ss, close, ',');
            getline(ss, adj_close, ',');
            getline(ss, volume, ',');

            IFintel::DataPoint dp;
            dp.date = date;
            dp.open = std::stof(open);
            dp.high = std::stof(high);
            dp.low = std::stof(low);
            dp.close = std::stof(close);

            data.push_back(dp);
        }
        file.close();
    } else {
        std::cout << "Unable to open file: " << filename << std::endl;
    }
    
    return data;
}

std::vector<IFintel::DataPoint> Controller::getFakeData(int data_size) {
    std::vector<IFintel::DataPoint> data;
    std::random_device rd;
    std::default_random_engine generator(rd());
    std::uniform_real_distribution<double> distribution(0.1,10);

    for (int i = 0; i < data_size; i++) {
        IFintel::DataPoint dp;
        dp.date = "";
        dp.open = distribution(generator);
        dp.high = distribution(generator);
        dp.low = distribution(generator);
        dp.close = distribution(generator);
        data.push_back(dp);
    }    
    return data;
}


int main() {
    std::cout << "Benchmarking CPU Implementation E2E\n";
    std::shared_ptr<IFintel> cpuFintel = std::make_shared<CPUFintel>(); 
    Controller c1(cpuFintel);
    std::cout << "SMA:\n";
    c1.benchmark("sma");
    std::cout << "EMA:\n";
    c1.benchmark("ema");
    std::cout << "RSI:\n";
    c1.benchmark("rsi");

    std::cout << "-----------------------------------------------------------\n";

    
    std::cout << "Benchmarking GPU Implementation E2E (Slow one -- ignore)\n";
    std::shared_ptr<IFintel> gpuFintel = std::make_shared<GPUFintel>(); 
    Controller c2(gpuFintel);
    std::cout << "SMA:\n";
    c2.benchmark("sma");
    std::cout << "EMA:\n";
    c2.benchmark("ema");
    std::cout << "RSI:\n";
    c2.benchmark("rsi");

    std::cout << "-----------------------------------------------------------\n";

    std::cout << "Benchmarking GPU Implementation E2E\n";
    std::shared_ptr<IFintel> gpuFintel3 = std::make_shared<GPUFintel>(); 
    Controller c4(gpuFintel3);
    std::cout << "SMA:\n";
    c4.benchmark("sma");
    std::cout << "EMA:\n";
    c4.benchmark("ema");
    std::cout << "RSI:\n";
    c4.benchmark("rsi");

    std::cout << "-----------------------------------------------------------\n";

    std::cout << "Benchmarking GPU Implementation Deep\n";
    std::shared_ptr<IFintel> gpuFintel2 = std::make_shared<TimedGPUFintel>(); 
    Controller c3(gpuFintel2);
    std::cout << "SMA:\n";
    c3.benchmark("sma");
    std::cout << "EMA:\n";
    c3.benchmark("ema");
    std::cout << "RSI:\n";
    c3.benchmark("rsi");
    return 0;
}


In [90]:
#@title Makefile
%%writefile Makefile

CXX=g++
CXXFLAGS=-g -std=c++17 -Wall
CUDA_PATH=/usr/local/cuda
NVCC=$(CUDA_PATH)/bin/nvcc
NVCCFLAGS=-O3 -arch=sm_60 -dc -std=c++14 -Xcompiler -Wall -Xcompiler -Wextra -I $(CUDA_PATH)/include
BIN=prog
CPP_SRC=$(filter-out GPUFintel.cpp, $(wildcard *.cpp))
CPP_OBJ=$(CPP_SRC:%.cpp=%.o)
CUDA_SRC=gpu_helpers.cu GPUFintel.cu TimedGPUFintel.cu 
CUDA_OBJ=$(CUDA_SRC:%.cu=%.o)

all: $(CPP_OBJ) $(CUDA_OBJ)
	$(NVCC) -arch=sm_60 -o $(BIN) $^ -L $(CUDA_PATH)/lib64 -lcudart -lpthread

%.o: %.cpp
	$(CXX) $(CXXFLAGS) -c $< -o $@

gpu_helpers.o: gpu_helpers.cu
	$(NVCC) $(NVCCFLAGS) -c $< -o $@

GPUFintel.o: GPUFintel.cu
	$(NVCC) $(NVCCFLAGS) -c $< -o $@

TimedGPUFintel.o: TimedGPUFintel.cu
	$(NVCC) $(NVCCFLAGS) -c $< -o $@

clean:
	rm -f *.o
	rm $(BIN)

Overwriting Makefile


# Execution

In [91]:
!make

g++ -g -std=c++17 -Wall -c controller.cpp -o controller.o
g++ -g -std=c++17 -Wall -c CPUFintel.cpp -o CPUFintel.o
/usr/local/cuda/bin/nvcc -O3 -arch=sm_60 -dc -std=c++14 -Xcompiler -Wall -Xcompiler -Wextra -I /usr/local/cuda/include -c gpu_helpers.cu -o gpu_helpers.o
/usr/local/cuda/bin/nvcc -O3 -arch=sm_60 -dc -std=c++14 -Xcompiler -Wall -Xcompiler -Wextra -I /usr/local/cuda/include -c GPUFintel.cu -o GPUFintel.o
/usr/local/cuda/bin/nvcc -O3 -arch=sm_60 -dc -std=c++14 -Xcompiler -Wall -Xcompiler -Wextra -I /usr/local/cuda/include -c TimedGPUFintel.cu -o TimedGPUFintel.o
/usr/local/cuda/bin/nvcc -arch=sm_60 -o prog controller.o CPUFintel.o gpu_helpers.o GPUFintel.o TimedGPUFintel.o -L /usr/local/cuda/lib64 -lcudart -lpthread


In [92]:
!./prog

Benchmarking CPU Implementation E2E
SMA:
[N:100] E2E Timing: 4.687000
[N:200] E2E Timing: 9.021000
[N:400] E2E Timing: 17.679000
[N:800] E2E Timing: 34.081000
[N:1600] E2E Timing: 67.423000
[N:3200] E2E Timing: 77.215000
[N:6400] E2E Timing: 153.292000
[N:12800] E2E Timing: 305.457000
[N:25600] E2E Timing: 635.962000
[N:51200] E2E Timing: 1271.480000
[N:102400] E2E Timing: 2797.993000
[N:204800] E2E Timing: 5462.019000
[N:409600] E2E Timing: 11279.952000
[N:819200] E2E Timing: 23571.768000
[N:1638400] E2E Timing: 45665.616000
[N:3276800] E2E Timing: 89110.964000
[N:6553600] E2E Timing: 260235.100000
EMA:
[N:100] E2E Timing: 2.604000
[N:200] E2E Timing: 5.381000
[N:400] E2E Timing: 9.922000
[N:800] E2E Timing: 19.272000
[N:1600] E2E Timing: 38.390000
[N:3200] E2E Timing: 75.612000
[N:6400] E2E Timing: 151.204000
[N:12800] E2E Timing: 308.604000
[N:25600] E2E Timing: 621.600000
[N:51200] E2E Timing: 1295.692000
[N:102400] E2E Timing: 2639.243000
[N:204800] E2E Timing: 5213.206000
[N:4096